## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [4]:
Instructions = "You are a research assistant. Given a search term, you will search the web for that term \
produce a concise summary of the results.The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search Agent", 
    instructions=Instructions, 
    tools=[WebSearchTool(search_context_size='low')], 
    model="gpt-4o-mini", model_settings=ModelSettings(tool_choice="required")
    )

In [5]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems.

**Agent Lightning** is a flexible framework that enables reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods. By formulating agent execution as a Markov decision process, it introduces a hierarchical RL algorithm, LightningRL, facilitating complex interaction logic handling. Experiments across tasks like text-to-SQL and retrieval-augmented generation demonstrate its potential for real-world agent training and deployment. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

**Polymorphic Combinatorial Framework (PCF)** leverages LLMs and mathematical frameworks to guide the design of adaptive AI agents. It enables real-time parameter reconfiguration through combinatorial spaces, allowing agents to dynamically adapt their behavioral traits. Grounded in combinatorial logic and fuzzy set theory, PCF defines a multidimensional parameter space (SPARK) to capture agent behaviors. By parameterizing complex domains and conducting extensive simulations, it reveals trends in agent adaptability and performance across varying complexity levels. PCF supports scalable, dynamic, explainable, and ethical AI applications in domains like customer service, healthcare, and robotics. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

**GoalfyMax** is a protocol-driven framework for end-to-end multi-agent collaboration. It introduces a standardized Agent-to-Agent (A2A) communication layer based on the Model Context Protocol (MCP), allowing independent agents to coordinate through asynchronous, protocol-compliant interactions. Incorporating the Experience Pack (XP) architecture, it preserves task rationales and execution traces, enabling structured knowledge retention and continual learning. Features like multi-turn contextual dialogue, long-short term memory modules, and dynamic safety validation support robust, real-time strategy adaptation. Empirical results on complex task orchestration benchmarks demonstrate superior adaptability, coordination, and experience reuse compared to baseline frameworks. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

Additionally, **Google Antigravity** is an AI-powered integrated development environment (IDE) developed by Google, designed for prioritizing AI agents platform for software development. Announced on November 18, 2025, Antigravity enables developers to delegate complex coding tasks to autonomous AI agents powered primarily by Google's Gemini 3 Pro, Gemini 3 Deep Think, and Gemini 3 Flash models. The platform introduces an "agent-first" paradigm, featuring an editor view with an agent sidebar and a manager view for orchestrating multiple agents working in parallel across workspaces. Agents generate "Artifacts"—verifiable deliverables such as task lists, implementation plans, screenshots, and browser recordings—rather than raw tool calls, enhancing trust and transparency. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Google_Antigravity?utm_source=openai))

These frameworks represent significant advancements in AI agent development, offering diverse approaches to creating intelligent, adaptable, and collaborative systems. 

In [6]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [7]:
message = "Create a health and wellness plan for a 37 year old man looking to improve his fitness and diet."

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find tailored health and wellness plans for middle-aged men.', query='fitness and wellness plan for 37 year old man'), WebSearchItem(reason='To gather information on nutrition guidelines and diet plans suited for health improvement.', query='diet plans for men 37 years old'), WebSearchItem(reason='To explore effective fitness routines and exercises for improving physical health.', query='best exercises for men over 30')]


In [8]:
@function_tool
def send_email(body: str, subject: str) -> Dict[str, str]:
    """ Send out an HTML email with the given body and subject to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("sabapathyk@outlook.com")
    to_email = To("sabapathy.ksp@gmail.com")
    content = Content("text/html", body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [9]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)


In [10]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [11]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [12]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [13]:
query = "Create a health and wellness plan for a 37 year old man looking to improve his fitness and diet. \
Also include mental wellness and stress management techniques. \
Include any supplement recommendations and a weekly exercise plan."

with trace("Deep Research Agent"):
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)

Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent


In [18]:
await send_email(report)

Writing email...
Email sent


ReportData(short_summary='A comprehensive health and wellness plan for a 37-year-old man seeking to improve fitness, diet, and mental wellness includes a balanced exercise regimen, nutritious meal guidelines, and stress management techniques. Key components consist of strength and cardio workouts, a whole-foods-based diet, and supplements like omega-3s and ashwagandha.', markdown_report="# Health and Wellness Plan for a 37-Year-Old Man\n\n## Table of Contents\n1. [Introduction](#introduction)\n2. [Fitness Improvement: Exercise Plan](#fitness-improvement-exercise-plan)\n   - [Weekly Workout Structure](#weekly-workout-structure)\n3. [Nutritional Guidelines](#nutritional-guidelines)\n   - [Sample Meal Plan](#sample-meal-plan)\n4. [Mental Wellness and Stress Management](#mental-wellness-and-stress-management)\n   - [Stress Management Techniques](#stress-management-techniques)\n   - [Supplement Recommendations](#supplement-recommendations)\n5. [Conclusion](#conclusion)\n\n## Introduction\nI

In [19]:
# Save the report to a markdown file
with open("health_wellness_report.md", "w") as f:
    f.write(report.markdown_report)

In [ ]:
display(Markdown( report.markdown_report))

# Health and Wellness Plan for a 37-Year-Old Man

## Table of Contents
1. [Introduction](#introduction)
2. [Fitness Improvement: Exercise Plan](#fitness-improvement-exercise-plan)
   - [Weekly Workout Structure](#weekly-workout-structure)
3. [Nutritional Guidelines](#nutritional-guidelines)
   - [Sample Meal Plan](#sample-meal-plan)
4. [Mental Wellness and Stress Management](#mental-wellness-and-stress-management)
   - [Stress Management Techniques](#stress-management-techniques)
   - [Supplement Recommendations](#supplement-recommendations)
5. [Conclusion](#conclusion)

## Introduction
In today’s fast-paced world, maintaining health and wellness is crucial, particularly as we age. For a 37-year-old man, the importance of a holistic approach to fitness, nutrition, and mental well-being cannot be overstated. This report outlines a comprehensive health and wellness plan targeting physical fitness improvements, dietary modifications, and mental wellness strategies.

## Fitness Improvement: Exercise Plan
### Weekly Workout Structure
A well-structured fitness plan plays a pivotal role in any health improvement effort. The following is a suggested weekly workout structure that focuses on strength training, cardiovascular exercise, and mobility work:

**1. Strength Training (3-4 Sessions/Week):**  
Strength training is vital for building muscle, supporting metabolism, and enhancing overall physical performance. Recommended exercises include:
- **Compound Movements:** Squats, deadlifts, bench presses, and overhead presses should be performed 3-4 sets of 8-12 repetitions each, focusing on progressive overload. This strategy gradually increases the weight for better results.  
- **Bodyweight Exercises:** Push-ups, pull-ups, and planks can also be included to develop functional strength without any equipment.

**2. Cardiovascular Exercise (2-3 Sessions/Week):**  
Integrating cardiovascular workouts enhances heart health and improves endurance. Suggested activities are:
- **Steady-State Cardio:** Engage in activities like jogging, cycling, or swimming for 20-30 minutes at a consistent pace.  
- **Interval Training:** Perform a cycle of high-intensity actions (like jumping jacks) for 10 seconds followed by a minute of rest, repeating this for 10-12 rounds.

**3. Mobility and Flexibility (Daily):**  
To maintain joint health and prevent injuries, it's essential to engage in daily mobility exercises focusing on major joints—the ankles, hips, shoulders, and lower back. Activities can include yoga poses or dynamic stretches tailored to enhance flexibility.

**4. Recovery:**  
Proper recovery is as important as the workouts themselves. Aim for 7-9 hours of sleep each night to facilitate muscle repair and overall recovery. 

## Nutritional Guidelines
A balanced and nutritious diet is critical for overall health and supports fitness endeavors. The following dietary recommendations should be incorporated:
1. **Macronutrient Ratios:**  
   - **Carbohydrates:** Constitute 45-65% of daily caloric intake, focusing on whole grains, fruits, and vegetables.  
   - **Proteins:** Aim for at least 1 gram of protein per pound of body weight to ensure muscle maintenance and recovery. Quality protein sources include lean meats, fish, tofu, and legumes. 
   - **Fats:** Healthy fats should contribute 20-35% of daily calories from sources such as avocados, nuts, and olive oil. 
2. **Micronutrients:**  
   - Ensure adequate intake of vitamins and minerals, particularly omega-3 fatty acids, necessary for heart health and inflammation reduction.
3. **Hydration:**  
   - Drink at least 2-3 liters of water daily, adjusting for physical activity and environmental conditions.

### Sample Meal Plan
Implementing a structured meal plan can help maintain optimal nutrition:
1. **Breakfast:** Scrambled eggs with spinach and tomatoes, whole-grain toast, mixed berries, and green tea.  
2. **Lunch:** Grilled chicken breast salad with mixed greens, cherry tomatoes, cucumbers, vinaigrette, and quinoa or brown rice.  
3. **Snack:** Greek yogurt with honey and almonds.  
4. **Dinner:** Baked salmon with lemon-dill sauce, steamed broccoli, and roasted sweet potatoes.

## Mental Wellness and Stress Management
Mental health is as important as physical health for overall well-being. Stress management techniques can significantly improve resilience and quality of life.
### Stress Management Techniques
1. **Meditation and Mindfulness:**  
   - Regular mindfulness practice, including meditation, can reduce stress levels. Aim for at least 10-20 minutes daily.
2. **Yoga:**  
   - Yoga combines movement and breathwork, promoting relaxation and mental flexibility. Regular practice can enhance both physical and mental health.
3. **Deep Breathing Exercises:**  
   - Simple breathing techniques can be used throughout the day to alleviate stress. A common method is inhaling deeply for four counts, holding for four counts, and exhaling for six counts.
4. **Nature Walks:**  
   - Spending time in nature has been shown to enhance mood and reduce stress. Aim for at least one nature walk weekly.

### Supplement Recommendations
Certain supplements can aid stress management and mental health:
1. **Ashwagandha:**  
   - An adaptogenic herb known for its ability to lower cortisol levels and reduce stress.
2. **L-theanine:**  
   - Found in green tea, this amino acid promotes relaxation without causing drowsiness.
3. **Magnesium:**  
   - Important for a calm nervous system and is often depleted during periods of stress.
4. **Omega-3 Fatty Acids:**  
   - Important for overall brain health and can reduce inflammation, thereby supporting mental health.

## Conclusion
Creating a comprehensive health and wellness plan for a 37-year-old man encompasses fitness improvement, nutritional modifications, and mental wellness strategies. By adhering to a structured exercise plan, maintaining a balanced diet, and incorporating stress management practices, one can hope to enhance overall fitness and well-being. Regular assessments and adjustments should be made to ensure continued progress and motivation. Finally, consultation with healthcare professionals before beginning any new health or dietary regimen is advised to tailor the plan to individual needs.

---